In [1]:
import xarray as xr
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import json

In [ ]:
# merge all "x_leadtimes_allmodels.nc" together

In [2]:
# Open the datasets
r = xr.open_dataset("Tropics_PearsonR_leadtimes_allmodels.nc")
rmse = xr.open_dataset("Tropics_RMSE_leadtimes_allmodels.nc")
mae = xr.open_dataset("Tropics_MAE_leadtimes_allmodels.nc")
mbe = xr.open_dataset("Tropics_MBE_leadtimes_allmodels.nc")

# List of shared variables
variables = list(set(r.data_vars) & set(rmse.data_vars) & set(mae.data_vars) & set(mbe.data_vars))

# Build a new dataset with a 'metric' dimension
combined_vars = {}
for var in variables:
    combined_vars[var] = xr.concat(
        [r[var].expand_dims(metric=["r"]),
         rmse[var].expand_dims(metric=["rmse"]),
         mae[var].expand_dims(metric=["mae"]),
         mbe[var].expand_dims(metric=["mbe"]),
        ],
        dim="metric"
    )

# Combine all variables into one Dataset
combined = xr.Dataset(combined_vars)

combined.to_netcdf("Tropics_R_RMSE_MAE_MBE_leadtimes_allmodels.nc")

# Final structure
print(combined)

<xarray.Dataset> Size: 256kB
Dimensions:  (metric: 4, time: 41, month: 13, model: 3)
Coordinates:
  * metric   (metric) object 32B 'r' 'rmse' 'mae' 'mbe'
  * time     (time) float64 328B 0.0 1.0 2.0 3.0 4.0 ... 37.0 38.0 39.0 40.0
  * month    (month) int64 104B 1 2 3 4 5 6 7 8 9 10 11 12 13
  * model    (model) <U6 72B 'marsai' 'gc' 'marsfc'
Data variables:
    u10      (metric, model, month, time) float64 51kB nan nan ... -0.1077
    v10      (metric, model, month, time) float64 51kB nan nan ... 0.06194
    msl      (metric, model, month, time) float64 51kB nan nan ... -0.4517
    t2m      (metric, model, month, time) float64 51kB nan nan ... -0.06956
    q        (metric, model, month, time) float64 51kB nan nan ... 0.1589 0.2576


In [ ]:
# turn the "R_RMSE_MAE_MBE_leadtimes_allmodels.nc" into a json file

In [3]:
# Load dataset
ds = xr.open_dataset("Tropics_R_RMSE_MAE_MBE_leadtimes_allmodels.nc")
# Drop the 'number' coordinate if it exists
if "number" in ds.coords:
    ds = ds.drop_vars("number")

# Convert to xarray.Dataset → pandas.DataFrame
df = ds.to_dataframe().reset_index()


# Convert DataFrame to a list of dictionaries (good for plotting)
data = df.to_dict(orient="records")

# Save to JSON
with open("Tropics_R_RMSE_MAE_MBE_leadtimes_allmodels.json", "w") as f:
    json.dump(data, f, indent=2)

print("Saved as Tropics_R_RMSE_MAE_MBE_leadtimes_allmodels.json")

Saved as Tropics_R_RMSE_MAE_MBE_leadtimes_allmodels.json
